In [14]:
def get_current_dir():
    import os
    from pathlib import Path
    try:
        return Path(__file__).parent.absolute()
    except NameError:
        return Path(os.getcwd())

path = get_current_dir().parent / "data" / "results.csv"

import pandas as pd
df = pd.read_csv(path)

# experiments from paper:
# - alpha_ent: 0.30 0.40
# - k: 20 50 100 200 (default=500 is not used in paper)
# - alpha_word: 0.2 0.3 0.4 0.5

# common values for undefined parameters:
# - n_topics: 10 20 50 100
# - n_neighbours: 5 10 20 500

missing = 0
for n_topics in [10, 20, 50, 100]:
    for n_neighbours in [5, 10, 20, 500]:
        for alpha_ent in [0.30, 0.40]:
            for k in [20, 50, 100, 200]:
                for alpha_word in [0.2, 0.3, 0.4, 0.5]:
                    mask = (df.n_topics == n_topics) & (df.n_neighbours == n_neighbours) & (df.alpha_ent == alpha_ent) & (df.k == k) & (df.alpha_word == alpha_word)
                    if not df[mask].empty:
                        continue
                    missing += 1
                    print(f"missing: n_topics={n_topics} n_neighbours={n_neighbours} alpha_ent={alpha_ent} k={k} alpha_word={alpha_word}")
cycle_time_s = 600
print(f"estimated time left: {missing * cycle_time_s // 3600}h {missing * cycle_time_s % 3600 // 60}m {missing * cycle_time_s % 60}s")

missing: n_topics=100 n_neighbours=10 alpha_ent=0.4 k=200 alpha_word=0.3
missing: n_topics=100 n_neighbours=10 alpha_ent=0.4 k=200 alpha_word=0.4
missing: n_topics=100 n_neighbours=10 alpha_ent=0.4 k=200 alpha_word=0.5
missing: n_topics=100 n_neighbours=20 alpha_ent=0.4 k=200 alpha_word=0.2
missing: n_topics=100 n_neighbours=20 alpha_ent=0.4 k=200 alpha_word=0.3
missing: n_topics=100 n_neighbours=20 alpha_ent=0.4 k=200 alpha_word=0.4
missing: n_topics=100 n_neighbours=20 alpha_ent=0.4 k=200 alpha_word=0.5
missing: n_topics=100 n_neighbours=500 alpha_ent=0.4 k=200 alpha_word=0.2
missing: n_topics=100 n_neighbours=500 alpha_ent=0.4 k=200 alpha_word=0.3
missing: n_topics=100 n_neighbours=500 alpha_ent=0.4 k=200 alpha_word=0.4
missing: n_topics=100 n_neighbours=500 alpha_ent=0.4 k=200 alpha_word=0.5
estimated time left: 1h 50m 0s
